# 1 rate

In [1]:
## make_radialprofiles_rate.py

"""
Iterate through Instrument: mos1, mos2, pn & sum up the instruments

Iterate through the source
Iterate through savepath
Iterate through bkgfrac cheese mask
Iterate through elo, ehi


"""
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os

def radial_prof(rad_bins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE):
    """
    This function plot the radial profile from xmm individual instrument data

    """

    import numpy as np
    from astropy.io import fits
    import pandas as pd
    import os
    from astropy.wcs import WCS
    from astropy.coordinates import SkyCoord

    regf = pd.read_csv(center_file)


    ra = regf[regf['ID']==id]['RA']
    dec = regf[regf['ID']==id]['DEC']


    # convert the fk5 coords to pixel coordinates
    dat_widths = fits.getdata(DATFILE).shape
    x,y = np.meshgrid(range(0, dat_widths[0]), range(0, dat_widths[1]))
    wcs = WCS(fits.getheader(DATFILE))
    RA,DEC = wcs.pixel_to_world_values(x,y)
    distance = SkyCoord(ra=ra,dec=dec,unit='deg').separation(SkyCoord(ra=RA,dec=DEC,unit='deg')).arcmin
    # ax.imshow(distance)

    cts_map = fits.getdata(DATFILE)*fits.getdata(CHEESEFILE)
    bkg_map = fits.getdata(NXBFILE)*fits.getdata(CHEESEFILE)
    exp_map = fits.getdata(EXPFILE)*fits.getdata(CHEESEFILE)
    msk_map = fits.getdata(CHEESEFILE)

    CTS, r_bins = np.histogram(distance.flatten(),weights=cts_map.flatten(),bins=rad_bins)

    BKG, r_bins = np.histogram(distance.flatten(),weights=bkg_map.flatten(),bins=rad_bins)

    EXP, r_bins = np.histogram(distance.flatten(),weights=exp_map.flatten(),bins=rad_bins)

    Npixel, r_bins = np.histogram(distance.flatten(),weights=msk_map.flatten(),bins=rad_bins)
    

    return CTS, BKG, EXP, Npixel


import numpy as np
import seaborn as sns
sns.set_palette("colorblind")

def make_plots(ax, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins):
    # Plotting using Seaborn color palette
    ax.stairs(cts, rbins, label='CTS', color=sns.color_palette()[0])
    ax.stairs(net, rbins, label='NET', color=sns.color_palette()[1])
    ax.stairs(bkg, rbins, label='BKG', color=sns.color_palette()[2])
    ax.stairs(exp, rbins, label='EXP (arbitrary scale)', color=sns.color_palette()[3])

    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=cts, yerr=ctserr,
                fmt='none', alpha=0.7, color=sns.color_palette()[0])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=net, yerr=neterr,
                fmt='none', alpha=0.5, color=sns.color_palette()[1])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=bkg, yerr=bkgerr,
                fmt='none', alpha=0.5, color=sns.color_palette()[2])

    ax.legend()
    ax.set_ylabel(f'$\\rm cts/s/pixel^2$')
    ax.set_xlabel(f'r [arcmin]')
    ax.set_xlim(0,20)
    ax.set_ylim([0, np.nanmedian(net) * 6])
    



ELOs = [500]#, 2000, 500]
EHIs = [2000]#, 8000, 8000]

center_file = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/all_centers.csv'
rootpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP'

rbins = np.linspace(0,30,100)

# for dir in glob(f'{rootpath}/SDSSTG11844'): 
#     id = dir.split('/')[-1]
for id in ['SDSSTG12349', 'SDSSTG828']:
    dir = f'{rootpath}/{id}'
    print(id)
    datapath = f'{dir}/{id}_allfov/final_img'
    savepath = f'{rootpath}/figs/radial_profiles/{id}'
    os.makedirs(savepath, exist_ok=True)
    for e_idx in range(len(ELOs)):
        for bkgfrac in np.arange(0.5, 2.6, 0.1):
            cts_insts = np.zeros((3, len(rbins)-1))
            bkg_insts = np.zeros((3, len(rbins)-1))
            exp_insts = np.zeros((3, len(rbins)-1))
            n_pix_insts = np.zeros((3, len(rbins)-1))
            for i, inst in enumerate(['mos1*', 'mos2*', 'pn*']):
                DATFILE = glob(f'{datapath}/{inst}-fovimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                EXPFILE = glob(f'{datapath}/{inst}-expimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                NXBFILE = glob(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                CHEESEFILE = glob(f'{dir}/{inst}-cheeset_{bkgfrac:.1f}_09fov.fits')[0]

                CTS, BKG, EXP, N_PIX = radial_prof(rbins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE)
                
                cts = CTS / EXP
                net = (CTS - BKG) / EXP
                bkg = BKG / EXP
                exp = EXP / N_PIX/ (np.median(EXP)/np.median(N_PIX)) * np.nanmedian(net) / 2.5

                ctserr = np.sqrt(CTS) / EXP
                neterr = np.sqrt(CTS - BKG) / EXP
                bkgerr = np.sqrt(BKG) / EXP
                
                fig, axs = plt.subplots()
                try:
                    make_plots(axs, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins)
                except ValueError:
                    print(f'{id} have incompelete files. ')
                    break
                axs.set_title(f'{id}_{DATFILE.split("/")[-1].split(".")[0]}')
                plt.savefig(f'{savepath}/inst-radprof_rate_{DATFILE.split("/")[-1].split(".")[0]}_{bkgfrac:.1f}.png')
                







# 2 cts

In [ ]:
## make_radialprofiles_cts.py

from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os

def radial_prof(rad_bins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE):
    """
    This function plot the radial profile from xmm individual instrument data

    """

    import numpy as np
    from astropy.io import fits
    import pandas as pd
    import os
    from astropy.wcs import WCS
    from astropy.coordinates import SkyCoord

    regf = pd.read_csv(center_file)


    ra = regf[regf['ID']==id]['RA']
    dec = regf[regf['ID']==id]['DEC']


    # convert the fk5 coords to pixel coordinates
    dat_widths = fits.getdata(DATFILE).shape
    x,y = np.meshgrid(range(0, dat_widths[0]), range(0, dat_widths[1]))
    wcs = WCS(fits.getheader(DATFILE))
    RA,DEC = wcs.pixel_to_world_values(x,y)
    distance = SkyCoord(ra=ra,dec=dec,unit='deg').separation(SkyCoord(ra=RA,dec=DEC,unit='deg')).arcmin
    # ax.imshow(distance)

    cts_map = fits.getdata(DATFILE)*fits.getdata(CHEESEFILE)
    bkg_map = fits.getdata(NXBFILE)*fits.getdata(CHEESEFILE)
    exp_map = fits.getdata(EXPFILE)*fits.getdata(CHEESEFILE)
    msk_map = fits.getdata(CHEESEFILE)

    CTS, r_bins = np.histogram(distance.flatten(),weights=cts_map.flatten(),bins=rad_bins)

    BKG, r_bins = np.histogram(distance.flatten(),weights=bkg_map.flatten(),bins=rad_bins)

    EXP, r_bins = np.histogram(distance.flatten(),weights=exp_map.flatten(),bins=rad_bins)

    Npixel, r_bins = np.histogram(distance.flatten(),weights=msk_map.flatten(),bins=rad_bins)
    

    return CTS, BKG, EXP, Npixel


import numpy as np
import seaborn as sns
sns.set_palette("colorblind")

def make_plots(ax, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins):
    # Plotting using Seaborn color palette
    ax.stairs(cts, rbins, label='CTS', color=sns.color_palette()[0])
    ax.stairs(net, rbins, label='NET', color=sns.color_palette()[1])
    ax.stairs(bkg, rbins, label='BKG', color=sns.color_palette()[2])
    ax.stairs(exp, rbins, label='EXP (arbitrary scale)', color=sns.color_palette()[3])

    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=cts, yerr=ctserr,
                fmt='none', alpha=0.7, color=sns.color_palette()[0])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=net, yerr=neterr,
                fmt='none', alpha=0.5, color=sns.color_palette()[1])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=bkg, yerr=bkgerr,
                fmt='none', alpha=0.5, color=sns.color_palette()[2])

    ax.legend()
    
    ax.legend()
    ax.set_ylabel(f'$\\rm cts / pixel^2$')
    ax.set_xlabel(f'r [arcmin]')
    ax.set_ylim([0, np.nanmedian(net) * 6])
    ax.set_xlim(0,20)
    



ELOs = [500, 2000, 500]
EHIs = [2000, 8000, 8000]

center_file = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/all_centers.csv'
rootpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP'

rbins = np.linspace(0,30,100)

for dir in glob(f'{rootpath}/SDSSTG11844'): 
    id = dir.split('/')[-1]
    print(id)
    datapath = f'{dir}/{id}_allfov/final_img'
    savepath = f'{rootpath}/figs/radial_profiles/{id}'
    os.makedirs(savepath, exist_ok=True)
    for e_idx in range(len(ELOs)):
        for bkgfrac in np.arange(0.5, 2.6, 0.1):
            cts_insts = np.zeros((3, len(rbins)-1))
            bkg_insts = np.zeros((3, len(rbins)-1))
            exp_insts = np.zeros((3, len(rbins)-1))
            n_pix_insts = np.zeros((3, len(rbins)-1))
            for i, inst in enumerate(['mos1*', 'mos2*', 'pn*']):
                DATFILE = glob(f'{datapath}/{inst}-fovimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                EXPFILE = glob(f'{datapath}/{inst}-expimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                NXBFILE = glob(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                CHEESEFILE = glob(f'{dir}/{inst}-cheeset_{bkgfrac:.1f}_09fov.fits')[0]

                CTS, BKG, EXP, N_PIX = radial_prof(rbins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE)
                
                cts = CTS / N_PIX
                net = (CTS - BKG) / N_PIX
                bkg = BKG / N_PIX
                exp = EXP / N_PIX/ (np.median(EXP)/np.median(N_PIX)) * np.nanmedian(net) / 2.5

                ctserr = np.sqrt(CTS) / N_PIX
                neterr = np.sqrt(CTS - BKG) / N_PIX
                bkgerr = np.sqrt(BKG) / N_PIX


                fig, axs = plt.subplots()
                try:
                    make_plots(axs, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins)
                except ValueError:
                    print(f'{id} have incompelete files. ')
                    break
                axs.set_title(f'{id}_{DATFILE.split("/")[-1].split(".")[0]}')
                plt.savefig(f'{savepath}/inst-radprof_cts_{DATFILE.split("/")[-1].split(".")[0]}_{bkgfrac:.1f}.png')

            





# 3 plot 3 insts ctr 

In [ ]:
# Notice: remember to scale the different instruments by ecf to sum up the 3 instruments (can be arbitrary)!
## https://www.cosmos.esa.int/web/xmm-newton/epic-upper-limits
# Here we don't scale and sum. just plot them together. 


## make_radialprofiles_rate_all.py

"""
Iterate through Instrument: mos1, mos2, pn & sum up the instruments

Iterate through the source
Iterate through savepath
Iterate through bkgfrac cheese mask
Iterate through elo, ehi


"""
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os

def radial_prof(rad_bins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE):
    """
    This function plot the radial profile from xmm individual instrument data

    """

    import numpy as np
    from astropy.io import fits
    import pandas as pd
    import os
    from astropy.wcs import WCS
    from astropy.coordinates import SkyCoord

    regf = pd.read_csv(center_file)


    ra = regf[regf['ID']==id]['RA']
    dec = regf[regf['ID']==id]['DEC']


    # convert the fk5 coords to pixel coordinates
    dat_widths = fits.getdata(DATFILE).shape
    x,y = np.meshgrid(range(0, dat_widths[0]), range(0, dat_widths[1]))
    wcs = WCS(fits.getheader(DATFILE))
    RA,DEC = wcs.pixel_to_world_values(x,y)
    distance = SkyCoord(ra=ra,dec=dec,unit='deg').separation(SkyCoord(ra=RA,dec=DEC,unit='deg')).arcmin
    # ax.imshow(distance)

    cts_map = fits.getdata(DATFILE)*fits.getdata(CHEESEFILE)
    bkg_map = fits.getdata(NXBFILE)*fits.getdata(CHEESEFILE)
    exp_map = fits.getdata(EXPFILE)*fits.getdata(CHEESEFILE)
    msk_map = fits.getdata(CHEESEFILE)

    CTS, r_bins = np.histogram(distance.flatten(),weights=cts_map.flatten(),bins=rad_bins)

    BKG, r_bins = np.histogram(distance.flatten(),weights=bkg_map.flatten(),bins=rad_bins)

    EXP, r_bins = np.histogram(distance.flatten(),weights=exp_map.flatten(),bins=rad_bins)

    Npixel, r_bins = np.histogram(distance.flatten(),weights=msk_map.flatten(),bins=rad_bins)
    

    return CTS, BKG, EXP, Npixel


import numpy as np
import seaborn as sns
sns.set_palette("colorblind")

def make_plots(ax, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins, inst):
    # Plotting using Seaborn color palette
    # ax.stairs(cts, rbins, label='CTS', color=sns.color_palette()[0])
    ax.stairs(net, rbins, label=f'NET-{inst}', color=sns.color_palette()[1])
    # ax.stairs(bkg, rbins, label='BKG', color=sns.color_palette()[2])
    # ax.stairs(exp, rbins, label='EXP (arbitrary scale)', color=sns.color_palette()[3])

    # ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=cts, yerr=ctserr,
    #             fmt='none', alpha=0.7, color=sns.color_palette()[0])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=net, yerr=neterr,
                fmt='none', alpha=0.5, color=sns.color_palette()[1])
    
    # ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=bkg, yerr=bkgerr,
    #             fmt='none', alpha=0.5, color=sns.color_palette()[2])

    ax.legend()
    ax.set_ylabel(f'$\\rm cts/s/pixel^2$')
    ax.set_xlabel(f'r [arcmin]')
    ax.set_xlim(0,20)
    ax.set_ylim([0, np.nanmedian(net) * 3])
    



ELOs = [500, 2000, 500]
EHIs = [2000, 8000, 8000]

center_file = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/all_centers.csv'
rootpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP'

rbins = np.linspace(0,30,100)

for dir in glob(f'{rootpath}/SDSSTG11844'): 
    id = dir.split('/')[-1]
    print(id)
    datapath = f'{dir}/{id}_allfov/final_img'
    savepath = f'{rootpath}/figs/radial_profiles/{id}'
    os.makedirs(savepath, exist_ok=True)
    for e_idx in range(len(ELOs)):
        for bkgfrac in np.arange(0.5, 2.6, 0.1):
            cts_insts = np.zeros((3, len(rbins)-1))
            bkg_insts = np.zeros((3, len(rbins)-1))
            exp_insts = np.zeros((3, len(rbins)-1))
            n_pix_insts = np.zeros((3, len(rbins)-1))
            fig, axs = plt.subplots()
            for i, inst in enumerate(['mos1*', 'mos2*', 'pn*']):

                DATFILE = glob(f'{datapath}/{inst}-fovimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                EXPFILE = glob(f'{datapath}/{inst}-expimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                NXBFILE = glob(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                CHEESEFILE = glob(f'{dir}/{inst}-cheeset_{bkgfrac:.1f}_09fov.fits')[0]

                CTS, BKG, EXP, N_PIX = radial_prof(rbins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE)
                
                cts = CTS / EXP
                net = (CTS - BKG) / EXP
                bkg = BKG / EXP
                exp = EXP / N_PIX/ (np.median(EXP)/np.median(N_PIX)) * np.nanmedian(net) / 2.5

                ctserr = np.sqrt(CTS) / EXP
                neterr = np.sqrt(CTS - BKG) / EXP
                bkgerr = np.sqrt(BKG) / EXP
                
            
                try:
                    make_plots(axs, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins, inst)
                except ValueError:
                    print(f'{id} have incompelete files. ')
                    break

            axs.set_title(f'{id}_{ELOs[e_idx]}-{EHIs[e_idx]}_{bkgfrac:.1f}')
            plt.savefig(f'{savepath}/esas-radprof_rate_{ELOs[e_idx]}-{EHIs[e_idx]}_{bkgfrac:.1f}.png')




# 4 plot 3 insts cts

In [ ]:
## make_radialprofiles_cts_all.py

from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os

def radial_prof(rad_bins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE):
    """
    This function plot the radial profile from xmm individual instrument data

    """

    import numpy as np
    from astropy.io import fits
    import pandas as pd
    import os
    from astropy.wcs import WCS
    from astropy.coordinates import SkyCoord

    regf = pd.read_csv(center_file)


    ra = regf[regf['ID']==id]['RA']
    dec = regf[regf['ID']==id]['DEC']


    # convert the fk5 coords to pixel coordinates
    dat_widths = fits.getdata(DATFILE).shape
    x,y = np.meshgrid(range(0, dat_widths[0]), range(0, dat_widths[1]))
    wcs = WCS(fits.getheader(DATFILE))
    RA,DEC = wcs.pixel_to_world_values(x,y)
    distance = SkyCoord(ra=ra,dec=dec,unit='deg').separation(SkyCoord(ra=RA,dec=DEC,unit='deg')).arcmin
    # ax.imshow(distance)

    cts_map = fits.getdata(DATFILE)*fits.getdata(CHEESEFILE)
    bkg_map = fits.getdata(NXBFILE)*fits.getdata(CHEESEFILE)
    exp_map = fits.getdata(EXPFILE)*fits.getdata(CHEESEFILE)
    msk_map = fits.getdata(CHEESEFILE)

    CTS, r_bins = np.histogram(distance.flatten(),weights=cts_map.flatten(),bins=rad_bins)

    BKG, r_bins = np.histogram(distance.flatten(),weights=bkg_map.flatten(),bins=rad_bins)

    EXP, r_bins = np.histogram(distance.flatten(),weights=exp_map.flatten(),bins=rad_bins)

    Npixel, r_bins = np.histogram(distance.flatten(),weights=msk_map.flatten(),bins=rad_bins)
    

    return CTS, BKG, EXP, Npixel


import numpy as np
import seaborn as sns
sns.set_palette("colorblind")

def make_plots(ax, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins):
    # Plotting using Seaborn color palette
    # ax.stairs(cts, rbins, label='CTS', color=sns.color_palette()[0])
    # ax.stairs(net, rbins, label='NET', color=sns.color_palette()[1])
    ax.stairs(bkg, rbins, label='BKG', color=sns.color_palette()[2])
    # ax.stairs(exp, rbins, label='EXP (arbitrary scale)', color=sns.color_palette()[3])

    # ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=cts, yerr=ctserr,
    #             fmt='none', alpha=0.7, color=sns.color_palette()[0])
    
    # ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=net, yerr=neterr,
    #             fmt='none', alpha=0.5, color=sns.color_palette()[1])
    
    ax.errorbar(x=np.sqrt(rbins[:-1]*rbins[1:]), y=bkg, yerr=bkgerr,
                fmt='none', alpha=0.5, color=sns.color_palette()[2])

    ax.legend()
    
    ax.legend()
    ax.set_ylabel(f'$\\rm cts / pixel^2$')
    ax.set_xlabel(f'r [arcmin]')
    ax.set_ylim([0, np.nanmedian(net) * 3])
    ax.set_xlim(0,20)
    



ELOs = [500, 2000, 500]
EHIs = [2000, 8000, 8000]

center_file = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/all_centers.csv'
rootpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP'

rbins = np.linspace(0,30,100)

for dir in glob(f'{rootpath}/SDSSTG11844'): 
    id = dir.split('/')[-1]
    print(id)
    datapath = f'{dir}/{id}_allfov/final_img'
    savepath = f'{rootpath}/figs/radial_profiles/{id}'
    os.makedirs(savepath, exist_ok=True)
    for e_idx in range(len(ELOs)):
        for bkgfrac in np.arange(0.5, 2.6, 0.1):
            cts_insts = np.zeros((3, len(rbins)-1))
            bkg_insts = np.zeros((3, len(rbins)-1))
            exp_insts = np.zeros((3, len(rbins)-1))
            n_pix_insts = np.zeros((3, len(rbins)-1))
            fig, axs = plt.subplots()
            for i, inst in enumerate(['mos1*', 'mos2*', 'pn*']):
                DATFILE = glob(f'{datapath}/{inst}-fovimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                EXPFILE = glob(f'{datapath}/{inst}-expimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                NXBFILE = glob(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                CHEESEFILE = glob(f'{dir}/{inst}-cheeset_{bkgfrac:.1f}_09fov.fits')[0]

                CTS, BKG, EXP, N_PIX = radial_prof(rbins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE)
                
                cts = CTS / N_PIX
                net = (CTS - BKG) / N_PIX
                bkg = BKG / N_PIX
                exp = EXP / N_PIX/ (np.median(EXP)/np.median(N_PIX)) * np.nanmedian(net) / 2.5

                ctserr = np.sqrt(CTS) / N_PIX
                neterr = np.sqrt(CTS - BKG) / N_PIX
                bkgerr = np.sqrt(BKG) / N_PIX


                
                try:
                    make_plots(axs, cts, net, bkg, exp, ctserr, neterr, bkgerr, rbins)
                except ValueError:
                    print(f'{id} have incompelete files. ')
                    break

            axs.set_title(f'{id}_{ELOs[e_idx]}-{EHIs[e_idx]}_{bkgfrac:.1f}')
            plt.savefig(f'{savepath}/esas-radprof_cts_{ELOs[e_idx]}-{EHIs[e_idx]}_{bkgfrac:.1f}.png')

            


